In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
df_expert = pd.read_csv('/content/drive/My Drive/Text Project/Expert_Opinion.csv', header=None, encoding = 'unicode_escape')
df_expert.columns = ['Comments']

In [4]:
df_expert.head()

,Comments
0,The American healthcare system functions prett...
1,As Democrats debate the best way to achieve un...
2,"INEQUALITY, NOT VIOLENCE, IS KILLING AMERICANS..."
3,According to The Commonwealth Funds most rece...
4,Home / Publications / Issue Fact Sheets and Re...


In [5]:
# Import the Data Frame
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# nltk.download('all')
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set( stopwords.words('english'))
from spacy.lang.en import English
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import spacy 
import numpy as np
import pandas as pd

# Function to remove stopwords and replacing model names with brand names
# This function will be applied to all comments in our data frame

def filterReview(review):
    nlp = English()
    tokens = word_tokenize(review)
    word_str = ""

    # Removing numbers
    for w in tokens:
        try:
            float(w)
        
        # If not a number, we will consider the words
        except Exception as e:
            if w.lower() not in stop_words: 
                word_str = word_str + " " + w.lower()
        continue
    
    # Lemmatization
    lemmatized = []
    doc = nlp(' '.join(word_str.split()))
    for word in doc:
        lemmatized.append(word.lemma_)
    
    return lemmatized

# Backup fundtion to clean the data 
def removePunctuations(review):
    nlp = English()
    tokens = word_tokenize(review)
    stop_words_punct = set(stopwords.words('english') + list(punctuation))
    no_stopwords = [w.lower() for w in tokens if w.lower() not in stop_words_punct]
    word_str = ""
    for w in no_stopwords:
        # Removing numbers
        try:
            float(w)
        except Exception as e:
            word_str = word_str + " " + w
        continue
    
    # Lemmatization
    lemmatized = []
    doc = nlp(' '.join(word_str.split()))
    for word in doc:
        lemmatized.append(word.lemma_)
    
    return lemmatized

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
df_expert['Filter Comments NO punct'] = df_expert['Comments'].map(removePunctuations)
# df_redUS['Filtered Comments'] = df_redUS['Comments'].map(filterReview)

In [7]:
df_expert.head()

,Comments,Filter Comments NO punct
0,The American healthcare system functions prett...,"[american, healthcare, system, function, prett..."
1,As Democrats debate the best way to achieve un...,"[democrat, debate, well, way, achieve, univers..."
2,"INEQUALITY, NOT VIOLENCE, IS KILLING AMERICANS...","[inequality, violence, kill, americans, record..."
3,According to The Commonwealth Funds most rece...,"[accord, commonwealth, funds, recent, study, ..."
4,Home / Publications / Issue Fact Sheets and Re...,"[home, publication, issue, fact, sheet, report..."


## Topic Modelling

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus, common_dictionary

In [0]:
dictionary_expert = gensim.corpora.Dictionary(df_expert['Filter Comments NO punct'])
dictionary_expert.filter_extremes(no_below=1, no_above=0.7) #, keep_n=100000)
bow_corpus_before = [dictionary_expert.doc2bow(words) for words in df_expert['Filter Comments NO punct']]
lda_model_expert = gensim.models.LdaMulticore(bow_corpus_before, num_topics=6, id2word=dictionary_expert, passes=2, workers=2, chunksize=100, random_state=1000)

In [10]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_expert.print_topics())

[   (   0,
        '0.024*"spend" + 0.022*"u.s" + 0.011*"comparable" + 0.009*"average" + '
        '0.009*"gdp" + 0.008*"per" + 0.007*"growth" + 0.006*"rate" + '
        '0.005*"similar" + 0.005*"sector"'),
    (   1,
        '0.007*"healthcare" + 0.006*"plan" + 0.006*"\x97" + 0.005*"pay" + '
        '0.005*"coverage" + 0.004*"administration" + 0.004*"it\x92s" + '
        '0.004*"trump" + 0.004*"cost" + 0.004*"hospital"'),
    (   2,
        '0.008*"africa" + 0.007*"drug" + 0.006*"government" + 0.006*"hospital" '
        '+ 0.005*"cost" + 0.005*"physician" + 0.004*"\x96" + 0.004*"delivery" '
        '+ 0.004*"population" + 0.004*"pay"'),
    (   3,
        '0.015*"cost" + 0.011*"percent" + 0.010*"u.s" + 0.009*"coverage" + '
        '0.007*"plan" + 0.006*"pay" + 0.006*"drug" + 0.006*"government" + '
        '0.005*"hospital" + 0.005*"medicare"'),
    (   4,
        '0.011*"hospital" + 0.010*"healthcare" + 0.010*"\x95" + '
        '0.009*"government" + 0.006*"doctor" + 0.006*"plan" + 0.0

In [11]:
lda_model_expert = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_expert, passes=10, workers=2, chunksize=100, random_state=1000)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_expert.print_topics())

[   (   0,
        '0.035*"spend" + 0.032*"u.s" + 0.017*"comparable" + 0.013*"gdp" + '
        '0.012*"average" + 0.010*"per" + 0.010*"growth" + 0.008*"rate" + '
        '0.007*"similar" + 0.007*"person"'),
    (   1,
        '0.009*"administration" + 0.008*"plan" + 0.007*"coverage" + '
        '0.007*"trump" + 0.007*"medicaid" + 0.007*"requirement" + 0.006*"poor" '
        '+ 0.006*"new" + 0.006*"sick" + 0.006*"\x97"'),
    (   2,
        '0.010*"cost" + 0.010*"hospital" + 0.007*"drug" + 0.007*"physician" + '
        '0.007*"\x96" + 0.007*"healthcare" + 0.006*"government" + 0.006*"plan" '
        '+ 0.006*"pay" + 0.006*"provider"'),
    (   3,
        '0.013*"cost" + 0.011*"healthcare" + 0.011*"percent" + 0.010*"u.s" + '
        '0.009*"coverage" + 0.007*"\x95" + 0.007*"government" + 0.007*"would" '
        '+ 0.006*"pay" + 0.006*"plan"'),
    (   4,
        '0.013*"measure" + 0.008*"doctor" + 0.008*"include" + 0.007*"facility" '
        '+ 0.007*"hospital" + 0.006*"population" + 0.00

## Topic Modelling after removing punctuations

In [0]:
dictionary_expert = gensim.corpora.Dictionary(df_expert['Filter Comments NO punct'])
dictionary_expert.filter_extremes(no_below=1, no_above=0.7) #, keep_n=100000)
bow_corpus_before = [dictionary_expert.doc2bow(words) for words in df_expert['Filter Comments NO punct']]
lda_model_expert = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_expert, passes=10, workers=2, chunksize=100, random_state=1000)

In [13]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_expert.print_topics())

[   (   0,
        '0.035*"spend" + 0.032*"u.s" + 0.017*"comparable" + 0.013*"gdp" + '
        '0.012*"average" + 0.010*"per" + 0.010*"growth" + 0.008*"rate" + '
        '0.007*"similar" + 0.007*"person"'),
    (   1,
        '0.009*"administration" + 0.008*"plan" + 0.007*"coverage" + '
        '0.007*"trump" + 0.007*"medicaid" + 0.007*"requirement" + 0.006*"poor" '
        '+ 0.006*"new" + 0.006*"sick" + 0.006*"\x97"'),
    (   2,
        '0.010*"cost" + 0.010*"hospital" + 0.007*"drug" + 0.007*"physician" + '
        '0.007*"\x96" + 0.007*"healthcare" + 0.006*"government" + 0.006*"plan" '
        '+ 0.006*"pay" + 0.006*"provider"'),
    (   3,
        '0.013*"cost" + 0.011*"healthcare" + 0.011*"percent" + 0.010*"u.s" + '
        '0.009*"coverage" + 0.007*"\x95" + 0.007*"government" + 0.007*"would" '
        '+ 0.006*"pay" + 0.006*"plan"'),
    (   4,
        '0.013*"measure" + 0.008*"doctor" + 0.008*"include" + 0.007*"facility" '
        '+ 0.007*"hospital" + 0.006*"population" + 0.00

In [14]:
lda_model_expert = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_expert, passes=30, workers=2, chunksize=100, random_state=1000)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_expert.print_topics())

[   (   0,
        '0.036*"spend" + 0.032*"u.s" + 0.017*"comparable" + 0.014*"gdp" + '
        '0.012*"average" + 0.010*"per" + 0.010*"growth" + 0.008*"rate" + '
        '0.007*"person" + 0.007*"similar"'),
    (   1,
        '0.009*"administration" + 0.008*"plan" + 0.007*"coverage" + '
        '0.007*"trump" + 0.007*"medicaid" + 0.007*"requirement" + 0.006*"poor" '
        '+ 0.006*"new" + 0.006*"sick" + 0.006*"\x97"'),
    (   2,
        '0.010*"cost" + 0.010*"hospital" + 0.007*"drug" + 0.007*"physician" + '
        '0.007*"\x96" + 0.007*"healthcare" + 0.007*"government" + 0.006*"plan" '
        '+ 0.006*"pay" + 0.006*"provider"'),
    (   3,
        '0.013*"cost" + 0.012*"healthcare" + 0.011*"percent" + 0.010*"u.s" + '
        '0.009*"coverage" + 0.007*"\x95" + 0.007*"government" + 0.007*"would" '
        '+ 0.006*"pay" + 0.006*"plan"'),
    (   4,
        '0.013*"measure" + 0.008*"doctor" + 0.008*"include" + 0.008*"facility" '
        '+ 0.007*"hospital" + 0.006*"population" + 0.00

## Additional Stop Words removal

In [0]:
new_stopword_list = ['gt', '`', '-', '“', '-', '/', 'amp', '•', 'subreddit', '’', '\'s',"\'\'", "\'", '”', 'argument', 'amitheasshole', 'https', 'http']
def additionalStop(comment):
    return [w for w in comment if len(w)>3 and w not in new_stopword_list]

In [0]:
df_expert['filter V1'] = df_expert['Filter Comments NO punct'].map(additionalStop)

In [0]:
dictionary_expert = gensim.corpora.Dictionary(df_expert['filter V1'])
dictionary_expert.filter_extremes(no_below=1, no_above=0.7) #, keep_n=100000)
bow_corpus_before = [dictionary_expert.doc2bow(words) for words in df_expert['filter V1']]

In [0]:
lda_model_expert = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_expert, passes=10, workers=2, chunksize=100, random_state=1000)

In [19]:
lda_model_expert = gensim.models.LdaMulticore(bow_corpus_before, num_topics=3, id2word=dictionary_expert, passes=10, workers=2, chunksize=100, random_state=1000)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_expert.print_topics())

[   (   0,
        '0.015*"cost" + 0.015*"percent" + 0.009*"drug" + 0.009*"coverage" + '
        '0.008*"price" + 0.007*"healthcare" + 0.007*"plan" + 0.005*"premium" + '
        '0.005*"medicaid" + 0.005*"company"'),
    (   1,
        '0.013*"healthcare" + 0.012*"spend" + 0.010*"rate" + 0.009*"measure" + '
        '0.009*"death" + 0.007*"report" + 0.006*"include" + 0.005*"average" + '
        '0.005*"comparable" + 0.005*"suicide"'),
    (   2,
        '0.011*"hospital" + 0.010*"government" + 0.010*"cost" + 0.008*"plan" + '
        '0.007*"would" + 0.007*"healthcare" + 0.007*"physician" + '
        '0.006*"medicare" + 0.006*"coverage" + 0.005*"provider"')]


In [20]:
lda_model_expert = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_expert, passes=10, workers=2, chunksize=100, random_state=1000)
lda_model_expert.print_topics()

[(0,
  '0.020*"percent" + 0.013*"cost" + 0.011*"healthcare" + 0.011*"coverage" + 0.006*"government" + 0.006*"drug" + 0.006*"plan" + 0.006*"include" + 0.006*"medicaid" + 0.006*"premium"'),
 (1,
  '0.025*"spend" + 0.018*"measure" + 0.011*"comparable" + 0.010*"average" + 0.009*"three" + 0.008*"rate" + 0.007*"include" + 0.007*"performance" + 0.007*"growth" + 0.007*"income"'),
 (2,
  '0.012*"would" + 0.012*"healthcare" + 0.010*"government" + 0.009*"cost" + 0.008*"doctor" + 0.007*"africa" + 0.007*"plan" + 0.006*"hospital" + 0.006*"policy" + 0.006*"medicare"'),
 (3,
  '0.015*"cost" + 0.012*"healthcare" + 0.011*"drug" + 0.010*"price" + 0.008*"death" + 0.007*"company" + 0.006*"percent" + 0.006*"rate" + 0.005*"homicide" + 0.005*"suicide"'),
 (4,
  '0.013*"hospital" + 0.011*"physician" + 0.010*"plan" + 0.009*"cost" + 0.008*"government" + 0.008*"coverage" + 0.007*"unite" + 0.006*"provider" + 0.006*"germany" + 0.006*"payment"')]

In [21]:
lda_model_expert = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_expert, passes=50, workers=2, chunksize=100, random_state=1000)
lda_model_expert.print_topics()

[(0,
  '0.021*"percent" + 0.013*"cost" + 0.011*"healthcare" + 0.011*"coverage" + 0.006*"drug" + 0.006*"government" + 0.006*"plan" + 0.006*"medicaid" + 0.006*"premium" + 0.006*"include"'),
 (1,
  '0.025*"spend" + 0.018*"measure" + 0.011*"comparable" + 0.010*"average" + 0.009*"three" + 0.008*"rate" + 0.007*"include" + 0.007*"performance" + 0.007*"population" + 0.007*"income"'),
 (2,
  '0.013*"would" + 0.012*"healthcare" + 0.010*"government" + 0.009*"cost" + 0.008*"doctor" + 0.007*"africa" + 0.007*"plan" + 0.006*"policy" + 0.006*"hospital" + 0.006*"medicare"'),
 (3,
  '0.015*"cost" + 0.012*"healthcare" + 0.011*"drug" + 0.010*"price" + 0.008*"death" + 0.007*"company" + 0.006*"rate" + 0.006*"percent" + 0.005*"homicide" + 0.005*"suicide"'),
 (4,
  '0.013*"hospital" + 0.011*"physician" + 0.010*"plan" + 0.009*"cost" + 0.008*"government" + 0.008*"coverage" + 0.007*"unite" + 0.006*"provider" + 0.006*"payment" + 0.006*"germany"')]

In [0]:
lda_model_expert = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_expert, chunksize = 64, eval_every = 5, per_word_topics = 3, minimum_probability = 0.1, passes=25, random_state=23)

In [23]:
lda_model_expert.print_topics()

[(0,
  '0.032*"spend" + 0.015*"comparable" + 0.011*"average" + 0.010*"rate" + 0.009*"region" + 0.009*"growth" + 0.006*"person" + 0.006*"report" + 0.006*"africa" + 0.006*"period"'),
 (1,
  '0.018*"healthcare" + 0.009*"death" + 0.009*"americans" + 0.008*"europe" + 0.008*"policy" + 0.008*"suicide" + 0.008*"homicide" + 0.007*"government" + 0.007*"coverage" + 0.007*"rate"'),
 (2,
  '0.014*"cost" + 0.013*"drug" + 0.010*"healthcare" + 0.008*"price" + 0.007*"company" + 0.007*"field" + 0.007*"pauly" + 0.006*"plan" + 0.006*"would" + 0.005*"accord"'),
 (3,
  '0.021*"percent" + 0.012*"cost" + 0.012*"coverage" + 0.011*"healthcare" + 0.008*"plan" + 0.007*"include" + 0.007*"measure" + 0.006*"medicaid" + 0.006*"premium" + 0.005*"average"'),
 (4,
  '0.013*"hospital" + 0.011*"cost" + 0.010*"government" + 0.008*"physician" + 0.008*"plan" + 0.006*"medicare" + 0.006*"provider" + 0.006*"coverage" + 0.005*"patient" + 0.005*"unite"')]

In [0]:
# df_redUS.to_csv('RedditUSA.csv')

## Preprocessing to remove the most common trivial words

In [0]:
from collections import Counter
s = df_expert['filter V1']
text = s.apply(pd.Series).stack().reset_index(drop=True)
word_counts = Counter(text)
common_words = word_counts.most_common()

In [0]:
df2 = pd.DataFrame(common_words)
df2.to_csv('/content/drive/My Drive/Text Project/df2.csv')

In [0]:
common_stops = ['people','much','well','time', 'would', 'like', 'make', 'work','long','still','even','give', 'good', 'think', 'want', 'right', 'thing', 'year', 'take', 'also', 'know', 'come', 'every', 'life', 'look', 'have', 'will', 'tell', 'believe', 'talk', 'seem', 'since', 'show', 'else','like'] 

In [0]:
new_stopword_list_v1 = ['//www.guardian.com', 'people','much','well','time', 'would', 'like', 'make', 'work','long','still','even','give', 'good', 'think', 'want', 'right', 'thing', 'year', 'take', 'also', 'know', 'come', 'every', 'life', 'look', 'have', 'will', 'tell', 'believe', 'talk', 'seem', 'since', 'show', 'else','like'] 

def additionalStop_V1(comment):
    return [w for w in comment if len(w)>3 and w not in new_stopword_list_v1]

df_expert['filter V2'] = df_expert['filter V1'].map(additionalStop_V1)

In [29]:
df_expert.head()

,Comments,Filter Comments NO punct,filter V1,filter V2
0,The American healthcare system functions prett...,"[american, healthcare, system, function, prett...","[american, healthcare, system, function, prett...","[american, healthcare, system, function, prett..."
1,As Democrats debate the best way to achieve un...,"[democrat, debate, well, way, achieve, univers...","[democrat, debate, well, achieve, universal, c...","[democrat, debate, achieve, universal, coverag..."
2,"INEQUALITY, NOT VIOLENCE, IS KILLING AMERICANS...","[inequality, violence, kill, americans, record...","[inequality, violence, kill, americans, record...","[inequality, violence, kill, americans, record..."
3,According to The Commonwealth Funds most rece...,"[accord, commonwealth, funds, recent, study, ...","[accord, commonwealth, funds, recent, study, ...","[accord, commonwealth, funds, recent, study, ..."
4,Home / Publications / Issue Fact Sheets and Re...,"[home, publication, issue, fact, sheet, report...","[home, publication, issue, fact, sheet, report...","[home, publication, issue, fact, sheet, report..."


In [0]:
dictionary_expert = gensim.corpora.Dictionary(df_expert['filter V2'])
dictionary_expert.filter_extremes(no_below=1, no_above=0.5) #, keep_n=100000)
bow_corpus_before = [dictionary_expert.doc2bow(words) for words in df_expert['filter V2']]

In [31]:
lda_model_expert = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_expert, passes=50, workers=2, chunksize=100, random_state=1000)
lda_model_expert.print_topics()

[(0,
  '0.027*"healthcare" + 0.008*"pauly" + 0.008*"field" + 0.008*"sander" + 0.007*"note" + 0.007*"payer" + 0.006*"encourage" + 0.006*"payment" + 0.006*"single" + 0.006*"trillion"'),
 (1,
  '0.021*"healthcare" + 0.009*"policy" + 0.009*"medicare" + 0.008*"death" + 0.007*"europe" + 0.007*"homicide" + 0.007*"suicide" + 0.006*"citizen" + 0.006*"overdose" + 0.006*"america"'),
 (2,
  '0.017*"africa" + 0.008*"facility" + 0.007*"improve" + 0.007*"region" + 0.007*"standard" + 0.006*"sector" + 0.006*"report" + 0.006*"quality" + 0.005*"african" + 0.005*"disease"'),
 (3,
  '0.010*"physician" + 0.008*"unite" + 0.007*"medicaid" + 0.006*"program" + 0.006*"individual" + 0.006*"average" + 0.005*"payment" + 0.005*"germany" + 0.005*"medicare" + 0.005*"sickness"'),
 (4,
  '0.025*"percent" + 0.012*"healthcare" + 0.008*"premium" + 0.008*"price" + 0.007*"measure" + 0.006*"average" + 0.005*"oecd" + 0.005*"uninsured" + 0.005*"medicare" + 0.005*"worker"')]

## Summarization

In [0]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df_expert['Comments']:
  sentences.append(sent_tokenize(s))
sentences = [y for x in sentences for y in x] # flatten list

In [33]:
sentences[:5]

['The American healthcare system functions pretty differently than healthcare in Europe \x97 and most healthcare systems in other first world countries, for that matter.',
 'With the ongoing healthcare debate in America (from repealing and replacing the Affordable Care Act to Senator Sanders\x92 Medicare-For-All proposal), many people have begun to ask why we can\x92t have a system like Canada, the U.K., France, or most other European nations.',
 'In order to decide whether or not those types of systems would be suitable for America (a debate we will not delve into here), we first have to understand how healthcare in Europe works.',
 'How Healthcare in Europe Works\nGenerally speaking, most European nations (in addition to others around the world) have some type of universal healthcare.',
 'According to the definition provided by the World Health Organization (WHO), this means that everyone has equal access to quality healthcare that improves the health of patients and that seeking suc

In [34]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2019-12-01 07:22:40--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-12-01 07:22:41--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-12-01 07:22:41--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [0]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [0]:
# function to remove stopwords
def remove_stopwords_sentence(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [0]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords_sentence(r.split()) for r in clean_sentences]

In [0]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [0]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [0]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [0]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [44]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

A lot of people in the gig economy dont have employer health insurance. According to Pauly, those who signed up for coverage for this year may well be the core portion of the population that finds getting their health insurance through the exchanges a good deal because the great bulk of them get subsidies.

Heavy federal subsidies are designed to offset premium increases by insurers in the health care marketplace, or the health exchanges that individual states operate.
That means theres more money being generated in that industry than ever before. He noted that during the recent recession, health care was the bright spot in terms of hiring. Added Field: Health care in this country is a giant jobs program, and its immune to recessions.

ADDITIONAL READING
PUBLIC POLICY
What Changes Are Ahead for the Affordable Care Act?
In this sense, the delivery of health care is similar to that found in the United States where, for the most part, large numbers of employee groups, indepen